# 数据表转换

In [4]:
import codecs
import csv
import os
import dgl
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ast

## 等分插值采样函数

In [5]:
def tansRSSI(data):
    t = np.linspace(0,60,len(data))
    rssi = np.array(data)
    re_rssi = signal.resample(rssi,120)
    t1 = np.linspace(0,60,120,endpoint=True)
    return re_rssi.tolist()

## SINR 计算函数

In [6]:
def rssi_to_linear(rssi_dbm):
    return 10 ** (rssi_dbm / 10)

def linear_to_rssi(power_mw):
    return 10 * np.log10(power_mw)

def calculate_sinr(useful_rssi_dbm_list, interference_rssi_dbm_list, scale_factors):
    noise_rssi_dbm = -100
    noise_rssi_mw = rssi_to_linear(noise_rssi_dbm)
    total_useful_rssi_mw = sum(rssi_to_linear(rssi) for rssi in useful_rssi_dbm_list)
    total_interference_mw = 0
    
    for interference_rssi_dbm, scale in zip(interference_rssi_dbm_list, scale_factors):
        interference_rssi_mw = rssi_to_linear(interference_rssi_dbm)
        weighted_interference_mw = scale * interference_rssi_mw + (1 - scale) * noise_rssi_mw
        total_interference_mw += weighted_interference_mw
    
    sinr_mw = total_useful_rssi_mw / total_interference_mw
    sinr_db = linear_to_rssi(sinr_mw)
    
    return sinr_db


## 2ap的ap节点特征

In [ ]:
src_file_name = 'training_set_2ap_loc0_nav82'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
print(len(data))

for i in range(len(data)):
    if(i%2==0) :
        ap_dicts_list = []
        ap_dict1 = {}
        row = data[i]
        ap_dict1['type'] = 0
        ap_dict1['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        ap_dict1['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        ap_dict1['protocol'] = row['protocol']
        ap_dict1['bss_id'] = row['bss_id']
        ap_dict1['pd'] = row['pd']
        ap_dict1['ed'] = row['ed']
        ap_dict1['nav'] = row['nav']
        ap_dict1['eirp'] = row['eirp']
        ap_dict1['sinr'] = 0
        if (ap_dict1['protocol'] == 'udp') :
            ap_dict1['protocol'] = 1
        elif (ap_dict1['protocol'] == 'tcp') :
            ap_dict1['protocol'] = 2
        
        ap_dicts_list.append(ap_dict1)

        ap_dict = {}
        row = data[i+1]
        ap_dict['type'] = 0
        ap_dict['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        ap_dict['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        ap_dict['protocol'] = row['protocol']
        ap_dict['bss_id'] = row['bss_id']
        ap_dict['pd'] = row['pd']
        ap_dict['ed'] = row['ed']
        ap_dict['nav'] = row['nav']
        ap_dict['eirp'] = row['eirp']
        ap_dict['sinr'] = 0
        if (ap_dict['protocol'] == 'udp') :
            ap_dict['protocol'] = 1
        elif (ap_dict['protocol'] == 'tcp') :
            ap_dict['protocol'] = 2
        
        ap_dicts_list.append(ap_dict)
        print(ap_dicts_list)
        for j in range(120):
            folder_path = "./out/"+src_file_name+"/node_ap/test"+str(i//2)
            filename = folder_path + "/nodes_ap_" + str(j).zfill(3) + ".csv"
            
            os.makedirs(folder_path, exist_ok=True)
            
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=ap_dicts_list[0].keys())
                writer.writeheader()
                writer.writerows(ap_dicts_list)
        ap_dicts_list.clear()
        ap_dict1.clear()
        ap_dict.clear()

## 2ap的ap节点预测值 - 训练数据

In [ ]:
src_file_name = 'training_set_2ap_loc0_nav82'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
print(len(data))

for i in range(len(data)):
    if(i%2==0) :
        ap_dicts_list = []
        ap_dict1 = {}
        row = data[i]
        ap_dict1['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        ap_dict1['nss'] = float(row['nss'])
        ap_dict1['mcs'] = float(row['mcs'])
        ap_dict1['per'] = float(row['per'])
        ap_dict1['num_ampdu'] = float(row['num_ampdu'])
        ap_dict1['ppdu_dur'] = float(row['ppdu_dur'])
        ap_dict1['other_air_time'] = float(row['other_air_time'])
        ap_dict1['seq_time'] = float(row['seq_time'])
        ap_dict1['throughput'] = float(row['throughput'])
        ap_dicts_list.append(ap_dict1)

        ap_dict = {}
        row = data[i+1]
        ap_dict['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        ap_dict['nss'] = float(row['nss'])
        ap_dict['mcs'] = float(row['mcs'])
        ap_dict['per'] = float(row['per'])
        ap_dict['num_ampdu'] = float(row['num_ampdu'])
        ap_dict['ppdu_dur'] = float(row['ppdu_dur'])
        ap_dict['other_air_time'] = float(row['other_air_time'])
        ap_dict['seq_time'] = float(row['seq_time'])
        ap_dict['throughput'] = float(row['throughput'])
        ap_dicts_list.append(ap_dict)
        
        print(ap_dicts_list)
        for j in range(120):
            folder_path = "./out/"+src_file_name+"/node_ap_predict/test"+str(i//2)
            filename = folder_path + "/nodes_ap_predict_" + str(j).zfill(3) + ".csv"
            
            os.makedirs(folder_path, exist_ok=True)
            
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=ap_dicts_list[0].keys())
                writer.writeheader()
                writer.writerows(ap_dicts_list)
        ap_dicts_list.clear()
        ap_dict1.clear()
        ap_dict.clear()

## 2ap的ap节点预测值 - 测试数据

In [ ]:
src_file_name = 'test_set_2_2ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
print(len(data))

for i in range(len(data)):
    if(i%2==0) :
        ap_dicts_list = []
        ap_dict1 = {}
        row = data[i]
        ap_dict1['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        ap_dict1['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        if(row['nss']!="") : ap_dict1['nss'] = float(row['nss'])
        else : ap_dict1['nss'] = 0
        if(row['mcs']!="") : ap_dict1['mcs'] = float(row['mcs'])
        else : ap_dict1['mcs'] = 0
        if(row['per']!="") : ap_dict1['per'] = float(row['per'])
        else : ap_dict1['per'] = 0
        ap_dicts_list.append(ap_dict1)

        ap_dict = {}
        row = data[i+1]
        ap_dict['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        if(row['nss']!="") : ap_dict['nss'] = float(row['nss'])
        else : ap_dict['nss'] = 0
        if(row['mcs']!="") : ap_dict['mcs'] = float(row['mcs'])
        else : ap_dict['mcs'] = 0
        if(row['per']!="") : ap_dict['per'] = float(row['per'])
        else : ap_dict['per'] = 0
        ap_dicts_list.append(ap_dict)
        
        print(ap_dicts_list)
        for j in range(120):
            folder_path = "./out/"+src_file_name+"/node_ap_predict/test"+str(i//2)
            filename = folder_path + "/nodes_ap_predict_" + str(j).zfill(3) + ".csv"
            
            os.makedirs(folder_path, exist_ok=True)
            
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=ap_dicts_list[0].keys())
                writer.writeheader()
                writer.writerows(ap_dicts_list)
        ap_dicts_list.clear()
        ap_dict1.clear()
        ap_dict.clear()

## 3ap的ap节点特征

In [ ]:
src_file_name = 'test_set_1_3ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
print(len(data))

for i in range(len(data)):
    if(i%3==0) :
        ap_dicts_list = []
        ap_dict1 = {}
        row = data[i]
        ap_dict1['type'] = 0
        ap_dict1['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        ap_dict1['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        ap_dict1['protocol'] = row['protocol']
        ap_dict1['bss_id'] = row['bss_id']
        ap_dict1['pd'] = row['pd']
        ap_dict1['ed'] = row['ed']
        ap_dict1['nav'] = row['nav']
        ap_dict1['eirp'] = row['eirp']
        ap_dict1['sinr'] = 0
        if (ap_dict1['protocol'] == 'udp') :
            ap_dict1['protocol'] = 1
        elif (ap_dict1['protocol'] == 'tcp') :
            ap_dict1['protocol'] = 2
        
        ap_dicts_list.append(ap_dict1)

        ap_dict = {}
        row = data[i+1]
        ap_dict['type'] = 0
        ap_dict['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        ap_dict['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        ap_dict['protocol'] = row['protocol']
        ap_dict['bss_id'] = row['bss_id']
        ap_dict['pd'] = row['pd']
        ap_dict['ed'] = row['ed']
        ap_dict['nav'] = row['nav']
        ap_dict['eirp'] = row['eirp']
        ap_dict['sinr'] = 0
        if (ap_dict['protocol'] == 'udp') :
            ap_dict['protocol'] = 1
        elif (ap_dict['protocol'] == 'tcp') :
            ap_dict['protocol'] = 2

        ap_dicts_list.append(ap_dict)
        
        ap_dict2 = {}
        row = data[i+2]
        ap_dict2['type'] = 0
        ap_dict2['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        ap_dict2['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        ap_dict2['protocol'] = row['protocol']
        ap_dict2['bss_id'] = row['bss_id']
        ap_dict2['pd'] = row['pd']
        ap_dict2['ed'] = row['ed']
        ap_dict2['nav'] = row['nav']
        ap_dict2['eirp'] = row['eirp']
        ap_dict2['sinr'] = 0
        if (ap_dict2['protocol'] == 'udp') :
            ap_dict2['protocol'] = 1
        elif (ap_dict2['protocol'] == 'tcp') :
            ap_dict2['protocol'] = 2
        
        ap_dicts_list.append(ap_dict2)
        
        print(ap_dicts_list)
        for j in range(120):
            folder_path = "./out/"+src_file_name+"/node_ap/test"+str(i//3)
            filename = folder_path + "/nodes_ap_" + str(j).zfill(3) + ".csv"
            
            os.makedirs(folder_path, exist_ok=True)
            
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=ap_dicts_list[0].keys())
                writer.writeheader()
                writer.writerows(ap_dicts_list)
        ap_dicts_list.clear()
        ap_dict1.clear()
        ap_dict2.clear()
        ap_dict.clear()

## 3ap的ap节点预测值 - 训练数据

In [ ]:
src_file_name = 'test_set_2_3ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
print(len(data))

for i in range(len(data)):
    if(i%3==0) :
        ap_dicts_list = []
        ap_dict1 = {}
        row = data[i]
        ap_dicts_list = []
        ap_dict1 = {}
        row = data[i]
        ap_dict1['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        if(row['nss']!="") : ap_dict1['nss'] = float(row['nss'])
        else : ap_dict1['nss'] = 0
        if(row['mcs']!="") : ap_dict1['mcs'] = float(row['mcs'])
        else : ap_dict1['mcs'] = 0
        if(row['per']!="") : ap_dict1['per'] = float(row['per'])
        else : ap_dict1['per'] = 0
        ap_dict1['num_ampdu'] = float(row['num_ampdu'])
        ap_dict1['ppdu_dur'] = float(row['ppdu_dur'])
        ap_dict1['other_air_time'] = float(row['other_air_time'])
        ap_dict1['seq_time'] = float(row['seq_time'])
        ap_dict1['throughput'] = float(row['throughput'])
        ap_dicts_list.append(ap_dict1)

        ap_dict = {}
        row = data[i+1]
        ap_dict['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        if(row['nss']!="") : ap_dict['nss'] = float(row['nss'])
        else : ap_dict['nss'] = 0
        if(row['mcs']!="") : ap_dict['mcs'] = float(row['mcs'])
        else : ap_dict['mcs'] = 0
        if(row['per']!="") : ap_dict['per'] = float(row['per'])
        else : ap_dict['per'] = 0
        ap_dict1['num_ampdu'] = float(row['num_ampdu'])
        ap_dict1['ppdu_dur'] = float(row['ppdu_dur'])
        ap_dict1['other_air_time'] = float(row['other_air_time'])
        ap_dict1['seq_time'] = float(row['seq_time'])
        ap_dict1['throughput'] = float(row['throughput'])

        ap_dicts_list.append(ap_dict)
        
        ap_dict2 = {}
        row = data[i+2]
        ap_dict2['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        if(row['nss']!="") : ap_dict2['nss'] = float(row['nss'])
        else : ap_dict2['nss'] = 0
        if(row['mcs']!="") : ap_dict2['mcs'] = float(row['mcs'])
        else : ap_dict2['mcs'] = 0
        if(row['per']!="") : ap_dict2['per'] = float(row['per'])
        else : ap_dict2['per'] = 0
        ap_dict1['num_ampdu'] = float(row['num_ampdu'])
        ap_dict1['ppdu_dur'] = float(row['ppdu_dur'])
        ap_dict1['other_air_time'] = float(row['other_air_time'])
        ap_dict1['seq_time'] = float(row['seq_time'])
        ap_dict1['throughput'] = float(row['throughput'])
        
        ap_dicts_list.append(ap_dict2)
        
        print(ap_dicts_list)
        for j in range(120):
            folder_path = "./out/"+src_file_name+"/node_ap_predict/test"+str(i//3)
            filename = folder_path + "/nodes_ap_predict_" + str(j).zfill(3) + ".csv"
            
            os.makedirs(folder_path, exist_ok=True)
            
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=ap_dicts_list[0].keys())
                writer.writeheader()
                writer.writerows(ap_dicts_list)
        ap_dicts_list.clear()
        ap_dict1.clear()
        ap_dict2.clear()
        ap_dict.clear()

## 3ap的ap节点预测值 - 测试数据

In [ ]:
src_file_name = 'test_set_2_3ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
print(len(data))

for i in range(len(data)):
    if(i%3==0) :
        ap_dicts_list = []
        ap_dict1 = {}
        row = data[i]
        ap_dicts_list = []
        ap_dict1 = {}
        row = data[i]
        ap_dict1['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        if(row['nss']!="") : ap_dict1['nss'] = float(row['nss'])
        else : ap_dict1['nss'] = 0
        if(row['mcs']!="") : ap_dict1['mcs'] = float(row['mcs'])
        else : ap_dict1['mcs'] = 0
        if(row['per']!="") : ap_dict1['per'] = float(row['per'])
        else : ap_dict1['per'] = 0
        # ap_dict1['num_ampdu'] = float(row['num_ampdu'])
        # ap_dict1['ppdu_dur'] = float(row['ppdu_dur'])
        # ap_dict1['other_air_time'] = float(row['other_air_time'])
        # ap_dict1['seq_time'] = float(row['seq_time'])
        # ap_dict1['throughput'] = float(row['throughput'])
        ap_dicts_list.append(ap_dict1)

        ap_dict = {}
        row = data[i+1]
        ap_dict['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        if(row['nss']!="") : ap_dict['nss'] = float(row['nss'])
        else : ap_dict['nss'] = 0
        if(row['mcs']!="") : ap_dict['mcs'] = float(row['mcs'])
        else : ap_dict['mcs'] = 0
        if(row['per']!="") : ap_dict['per'] = float(row['per'])
        else : ap_dict['per'] = 0
        # ap_dict1['num_ampdu'] = float(row['num_ampdu'])
        # ap_dict1['ppdu_dur'] = float(row['ppdu_dur'])
        # ap_dict1['other_air_time'] = float(row['other_air_time'])
        # ap_dict1['seq_time'] = float(row['seq_time'])
        # ap_dict1['throughput'] = float(row['throughput'])

        ap_dicts_list.append(ap_dict)
        
        ap_dict2 = {}
        row = data[i+2]
        ap_dict2['id'] = int(row['ap_id'][3:len(row['ap_id'])])
        if(row['nss']!="") : ap_dict2['nss'] = float(row['nss'])
        else : ap_dict2['nss'] = 0
        if(row['mcs']!="") : ap_dict2['mcs'] = float(row['mcs'])
        else : ap_dict2['mcs'] = 0
        if(row['per']!="") : ap_dict2['per'] = float(row['per'])
        else : ap_dict2['per'] = 0
        # ap_dict1['num_ampdu'] = float(row['num_ampdu'])
        # ap_dict1['ppdu_dur'] = float(row['ppdu_dur'])
        # ap_dict1['other_air_time'] = float(row['other_air_time'])
        # ap_dict1['seq_time'] = float(row['seq_time'])
        # ap_dict1['throughput'] = float(row['throughput'])
        
        ap_dicts_list.append(ap_dict2)
        
        print(ap_dicts_list)
        for j in range(120):
            folder_path = "./out/"+src_file_name+"/node_ap_predict/test"+str(i//3)
            filename = folder_path + "/nodes_ap_predict_" + str(j).zfill(3) + ".csv"
            
            os.makedirs(folder_path, exist_ok=True)
            
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=ap_dicts_list[0].keys())
                writer.writeheader()
                writer.writerows(ap_dicts_list)
        ap_dicts_list.clear()
        ap_dict1.clear()
        ap_dict2.clear()
        ap_dict.clear()

## 两节点的 ap-ap 边特征

In [ ]:
src_file_name = 'test_set_2_2ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
# print(len(data))

for i in range(len(data)):
    if(data[i]['ap_from_ap_0_max_ant_rssi'] != "") : data[i]['ap_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_max_ant_rssi']))
    if(data[i]['ap_from_ap_1_max_ant_rssi'] != "") : data[i]['ap_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_max_ant_rssi']))
    if(data[i]['ap_from_ap_0_mean_ant_rssi'] != "") : data[i]['ap_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_mean_ant_rssi']))
    if(data[i]['ap_from_ap_1_mean_ant_rssi'] != "") : data[i]['ap_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_mean_ant_rssi']))
    if(data[i]['ap_from_ap_0_sum_ant_rssi'] != "") : data[i]['ap_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_sum_ant_rssi']))
    if(data[i]['ap_from_ap_1_sum_ant_rssi'] != "") : data[i]['ap_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_sum_ant_rssi']))
    
    if(data[i]['sta_from_ap_0_max_ant_rssi'] != "") : data[i]['sta_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_max_ant_rssi']))
    if(data[i]['sta_from_ap_1_max_ant_rssi'] != "") : data[i]['sta_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_max_ant_rssi']))
    if(data[i]['sta_from_ap_0_mean_ant_rssi'] != "") : data[i]['sta_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_mean_ant_rssi']))
    if(data[i]['sta_from_ap_1_mean_ant_rssi'] != "") : data[i]['sta_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_mean_ant_rssi']))
    if(data[i]['sta_from_ap_0_sum_ant_rssi'] != "") : data[i]['sta_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_sum_ant_rssi']))
    if(data[i]['sta_from_ap_1_sum_ant_rssi'] != "") : data[i]['sta_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_sum_ant_rssi']))

    if(data[i]['sta_to_ap_0_max_ant_rssi'] != "") : data[i]['sta_to_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_max_ant_rssi']))
    if(data[i]['sta_to_ap_1_max_ant_rssi'] != "") : data[i]['sta_to_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_max_ant_rssi']))
    if(data[i]['sta_to_ap_0_mean_ant_rssi'] != "") : data[i]['sta_to_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_mean_ant_rssi']))
    if(data[i]['sta_to_ap_1_mean_ant_rssi'] != "") : data[i]['sta_to_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_mean_ant_rssi']))
    if(data[i]['sta_to_ap_0_sum_ant_rssi'] != "") : data[i]['sta_to_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_sum_ant_rssi']))
    if(data[i]['sta_to_ap_1_sum_ant_rssi'] != "") : data[i]['sta_to_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_sum_ant_rssi']))

for i in range(len(data)):
    if(i%2==0) :
        # print(len(data[i]['ap_from_ap_0_sum_ant_rssi']))
        for j in range(120):         
            folder_path = "./out/"+src_file_name+"/edge_ap_ap/test"+str(i//2)
            filename = folder_path + "/ap_ap_" + str(j).zfill(3) + ".csv"
            edges_ap_ap = []
            
            dict01 = {}
            dict01['src']=0
            dict01['dst']=1
            dict01['rssi_sum']=data[i]['ap_from_ap_0_sum_ant_rssi'][j]
            dict01['rssi_mean']=data[i]['ap_from_ap_0_mean_ant_rssi'][j]
            dict01['rssi_max']=data[i]['ap_from_ap_0_max_ant_rssi'][j]
            edges_ap_ap.append(dict01)
            
            dict10 = {}
            dict10['src']=1
            dict10['dst']=0
            dict10['rssi_sum']=data[i+1]['ap_from_ap_1_sum_ant_rssi'][j]
            dict10['rssi_mean']=data[i+1]['ap_from_ap_1_mean_ant_rssi'][j]
            dict10['rssi_max']=data[i+1]['ap_from_ap_1_max_ant_rssi'][j]
            edges_ap_ap.append(dict10)

            os.makedirs(folder_path, exist_ok=True)
    
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=edges_ap_ap[0].keys())
                writer.writeheader()
                writer.writerows(edges_ap_ap)

## 三节点的 ap-ap 边特征

In [ ]:
src_file_name = 'test_set_2_3ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
# print(len(data))

for i in range(len(data)):
    if(data[i]['ap_from_ap_0_max_ant_rssi'] != "") : data[i]['ap_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_max_ant_rssi']))
    else : data[i]['ap_from_ap_0_max_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_1_max_ant_rssi'] != "") : data[i]['ap_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_max_ant_rssi']))
    else : data[i]['ap_from_ap_1_max_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_2_max_ant_rssi'] != "") : data[i]['ap_from_ap_2_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_2_max_ant_rssi']))
    else : data[i]['ap_from_ap_2_max_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_0_mean_ant_rssi'] != "") : data[i]['ap_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_mean_ant_rssi']))
    else : data[i]['ap_from_ap_0_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_1_mean_ant_rssi'] != "") : data[i]['ap_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_mean_ant_rssi']))
    else : data[i]['ap_from_ap_1_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_2_mean_ant_rssi'] != "") : data[i]['ap_from_ap_2_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_2_mean_ant_rssi']))
    else : data[i]['ap_from_ap_2_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_0_sum_ant_rssi'] != "") : data[i]['ap_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_sum_ant_rssi']))
    else : data[i]['ap_from_ap_0_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_1_sum_ant_rssi'] != "") : data[i]['ap_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_sum_ant_rssi']))
    else : data[i]['ap_from_ap_1_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_2_sum_ant_rssi'] != "") : data[i]['ap_from_ap_2_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_2_sum_ant_rssi']))
    else : data[i]['ap_from_ap_2_sum_ant_rssi'] = [-100.0]*120
    
    if(data[i]['sta_from_ap_0_max_ant_rssi'] != "") : data[i]['sta_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_max_ant_rssi']))
    if(data[i]['sta_from_ap_1_max_ant_rssi'] != "") : data[i]['sta_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_max_ant_rssi']))
    if(data[i]['sta_from_ap_2_max_ant_rssi'] != "") : data[i]['sta_from_ap_2_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_2_max_ant_rssi']))
    if(data[i]['sta_from_ap_0_mean_ant_rssi'] != "") : data[i]['sta_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_mean_ant_rssi']))
    if(data[i]['sta_from_ap_1_mean_ant_rssi'] != "") : data[i]['sta_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_mean_ant_rssi']))
    if(data[i]['sta_from_ap_2_mean_ant_rssi'] != "") : data[i]['sta_from_ap_2_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_2_mean_ant_rssi']))
    if(data[i]['sta_from_ap_0_sum_ant_rssi'] != "") : data[i]['sta_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_sum_ant_rssi']))
    if(data[i]['sta_from_ap_1_sum_ant_rssi'] != "") : data[i]['sta_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_sum_ant_rssi']))
    if(data[i]['sta_from_ap_2_sum_ant_rssi'] != "") : data[i]['sta_from_ap_2_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_2_sum_ant_rssi']))

    if(data[i]['sta_to_ap_0_max_ant_rssi'] != "") : data[i]['sta_to_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_max_ant_rssi']))
    if(data[i]['sta_to_ap_1_max_ant_rssi'] != "") : data[i]['sta_to_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_max_ant_rssi']))
    if(data[i]['sta_to_ap_2_max_ant_rssi'] != "") : data[i]['sta_to_ap_2_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_2_max_ant_rssi']))
    if(data[i]['sta_to_ap_0_mean_ant_rssi'] != "") : data[i]['sta_to_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_mean_ant_rssi']))
    if(data[i]['sta_to_ap_1_mean_ant_rssi'] != "") : data[i]['sta_to_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_mean_ant_rssi']))
    if(data[i]['sta_to_ap_2_mean_ant_rssi'] != "") : data[i]['sta_to_ap_2_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_2_mean_ant_rssi']))
    if(data[i]['sta_to_ap_0_sum_ant_rssi'] != "") : data[i]['sta_to_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_sum_ant_rssi']))
    if(data[i]['sta_to_ap_1_sum_ant_rssi'] != "") : data[i]['sta_to_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_sum_ant_rssi']))
    if(data[i]['sta_to_ap_2_sum_ant_rssi'] != "") : data[i]['sta_to_ap_2_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_2_sum_ant_rssi']))

for i in range(len(data)):
    if(i%3==0) :
        print(len(data[i+1]['ap_from_ap_0_sum_ant_rssi']))
        for j in range(120):         
            folder_path = "./out/"+src_file_name+"/edge_ap_ap/test"+str(i//3)
            filename = folder_path + "/ap_ap_" + str(j).zfill(3) + ".csv"
            edges_ap_ap = []
            
            dict01 = {}
            dict01['src']=0
            dict01['dst']=1
            dict01['rssi_sum']=data[i+1]['ap_from_ap_0_sum_ant_rssi'][j]
            dict01['rssi_mean']=data[i+1]['ap_from_ap_0_mean_ant_rssi'][j]
            dict01['rssi_max']=data[i+1]['ap_from_ap_0_max_ant_rssi'][j]
            edges_ap_ap.append(dict01)

            dict02 = {}
            dict02['src']=0
            dict02['dst']=2
            dict02['rssi_sum']=data[i]['ap_from_ap_0_sum_ant_rssi'][j]
            dict02['rssi_mean']=data[i]['ap_from_ap_0_mean_ant_rssi'][j]
            dict02['rssi_max']=data[i]['ap_from_ap_0_max_ant_rssi'][j]
            edges_ap_ap.append(dict02)
            
            dict10 = {}
            dict10['src']=1
            dict10['dst']=0
            dict10['rssi_sum']=data[i+2]['ap_from_ap_1_sum_ant_rssi'][j]
            dict10['rssi_mean']=data[i+2]['ap_from_ap_1_mean_ant_rssi'][j]
            dict10['rssi_max']=data[i+2]['ap_from_ap_1_max_ant_rssi'][j]
            edges_ap_ap.append(dict10)
            
            dict12 = {}
            dict12['src']=1
            dict12['dst']=2
            dict12['rssi_sum']=data[i]['ap_from_ap_1_sum_ant_rssi'][j]
            dict12['rssi_mean']=data[i]['ap_from_ap_1_mean_ant_rssi'][j]
            dict12['rssi_max']=data[i]['ap_from_ap_1_max_ant_rssi'][j]
            edges_ap_ap.append(dict12)
            
            dict20 = {}
            dict20['src']=2
            dict20['dst']=0
            dict20['rssi_sum']=data[i+2]['ap_from_ap_2_sum_ant_rssi'][j]
            dict20['rssi_mean']=data[i+2]['ap_from_ap_2_mean_ant_rssi'][j]
            dict20['rssi_max']=data[i+2]['ap_from_ap_2_max_ant_rssi'][j]
            edges_ap_ap.append(dict20)
            
            dict21 = {}
            dict21['src']=2
            dict21['dst']=1
            dict21['rssi_sum']=data[i+1]['ap_from_ap_2_sum_ant_rssi'][j]
            dict21['rssi_mean']=data[i+1]['ap_from_ap_2_mean_ant_rssi'][j]
            dict21['rssi_max']=data[i+1]['ap_from_ap_2_max_ant_rssi'][j]
            edges_ap_ap.append(dict21)

            os.makedirs(folder_path, exist_ok=True)
    
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=edges_ap_ap[0].keys())
                writer.writeheader()
                writer.writerows(edges_ap_ap)

## 两节点的 AP-STA 边属性

In [ ]:
src_file_name = 'test_set_2_2ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
# print(len(data))

for i in range(len(data)):
    if(data[i]['ap_from_ap_0_max_ant_rssi'] != "") : data[i]['ap_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_max_ant_rssi']))
    if(data[i]['ap_from_ap_1_max_ant_rssi'] != "") : data[i]['ap_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_max_ant_rssi']))
    if(data[i]['ap_from_ap_0_mean_ant_rssi'] != "") : data[i]['ap_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_mean_ant_rssi']))
    if(data[i]['ap_from_ap_1_mean_ant_rssi'] != "") : data[i]['ap_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_mean_ant_rssi']))
    if(data[i]['ap_from_ap_0_sum_ant_rssi'] != "") : data[i]['ap_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_sum_ant_rssi']))
    if(data[i]['ap_from_ap_1_sum_ant_rssi'] != "") : data[i]['ap_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_sum_ant_rssi']))
    
    if(data[i]['sta_from_ap_0_max_ant_rssi'] != "") : data[i]['sta_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_max_ant_rssi']))
    if(data[i]['sta_from_ap_1_max_ant_rssi'] != "") : data[i]['sta_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_max_ant_rssi']))
    if(data[i]['sta_from_ap_0_mean_ant_rssi'] != "") : data[i]['sta_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_mean_ant_rssi']))
    if(data[i]['sta_from_ap_1_mean_ant_rssi'] != "") : data[i]['sta_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_mean_ant_rssi']))
    if(data[i]['sta_from_ap_0_sum_ant_rssi'] != "") : data[i]['sta_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_sum_ant_rssi']))
    if(data[i]['sta_from_ap_1_sum_ant_rssi'] != "") : data[i]['sta_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_sum_ant_rssi']))

    if(data[i]['sta_to_ap_0_max_ant_rssi'] != "") : data[i]['sta_to_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_max_ant_rssi']))
    if(data[i]['sta_to_ap_1_max_ant_rssi'] != "") : data[i]['sta_to_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_max_ant_rssi']))
    if(data[i]['sta_to_ap_0_mean_ant_rssi'] != "") : data[i]['sta_to_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_mean_ant_rssi']))
    if(data[i]['sta_to_ap_1_mean_ant_rssi'] != "") : data[i]['sta_to_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_mean_ant_rssi']))
    if(data[i]['sta_to_ap_0_sum_ant_rssi'] != "") : data[i]['sta_to_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_sum_ant_rssi']))
    if(data[i]['sta_to_ap_1_sum_ant_rssi'] != "") : data[i]['sta_to_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_sum_ant_rssi']))

for i in range(len(data)):
    if(i%2==0) :
        # print(len(data[i]['ap_from_ap_0_sum_ant_rssi']))
        for j in range(120):         
            folder_path = "./out/"+src_file_name+"/edge_ap_sta/test"+str(i//2)
            filename = folder_path + "/ap_sta_" + str(j).zfill(3) + ".csv"
            edges_ap_sta = []
            
            dict00 = {}
            dict00['src']=0
            dict00['dst']=0
            dict00['rssi_sum']=data[i+1]['sta_from_ap_0_sum_ant_rssi'][j]
            dict00['rssi_mean']=data[i+1]['sta_from_ap_0_mean_ant_rssi'][j]
            dict00['rssi_max']=data[i+1]['sta_from_ap_0_max_ant_rssi'][j]
            edges_ap_sta.append(dict00)
            
            dict10 = {}
            dict10['src']=1
            dict10['dst']=0
            dict10['rssi_sum']=data[i+1]['sta_from_ap_1_sum_ant_rssi'][j]
            dict10['rssi_mean']=data[i+1]['sta_from_ap_1_mean_ant_rssi'][j]
            dict10['rssi_max']=data[i+1]['sta_from_ap_1_max_ant_rssi'][j]
            edges_ap_sta.append(dict10)

            dict01 = {}
            dict01['src']=0
            dict01['dst']=1
            dict01['rssi_sum']=data[i]['sta_from_ap_0_sum_ant_rssi'][j]
            dict01['rssi_mean']=data[i]['sta_from_ap_0_mean_ant_rssi'][j]
            dict01['rssi_max']=data[i]['sta_from_ap_0_max_ant_rssi'][j]
            edges_ap_sta.append(dict01)
            
            dict11 = {}
            dict11['src']=1
            dict11['dst']=1
            dict11['rssi_sum']=data[i]['sta_from_ap_1_sum_ant_rssi'][j]
            dict11['rssi_mean']=data[i]['sta_from_ap_1_mean_ant_rssi'][j]
            dict11['rssi_max']=data[i]['sta_from_ap_1_max_ant_rssi'][j]
            edges_ap_sta.append(dict11)

            os.makedirs(folder_path, exist_ok=True)
    
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=edges_ap_sta[0].keys())
                writer.writeheader()
                writer.writerows(edges_ap_sta)

## 三节点的AP-STA边特征

In [ ]:
src_file_name = 'test_set_2_3ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
# print(len(data))

for i in range(len(data)):
    if(data[i]['ap_from_ap_0_max_ant_rssi'] != "") : data[i]['ap_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_max_ant_rssi']))
    else : data[i]['ap_from_ap_0_max_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_1_max_ant_rssi'] != "") : data[i]['ap_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_max_ant_rssi']))
    else : data[i]['ap_from_ap_1_max_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_2_max_ant_rssi'] != "") : data[i]['ap_from_ap_2_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_2_max_ant_rssi']))
    else : data[i]['ap_from_ap_2_max_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_0_mean_ant_rssi'] != "") : data[i]['ap_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_mean_ant_rssi']))
    else : data[i]['ap_from_ap_0_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_1_mean_ant_rssi'] != "") : data[i]['ap_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_mean_ant_rssi']))
    else : data[i]['ap_from_ap_1_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_2_mean_ant_rssi'] != "") : data[i]['ap_from_ap_2_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_2_mean_ant_rssi']))
    else : data[i]['ap_from_ap_2_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_0_sum_ant_rssi'] != "") : data[i]['ap_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_sum_ant_rssi']))
    else : data[i]['ap_from_ap_0_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_1_sum_ant_rssi'] != "") : data[i]['ap_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_sum_ant_rssi']))
    else : data[i]['ap_from_ap_1_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_2_sum_ant_rssi'] != "") : data[i]['ap_from_ap_2_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_2_sum_ant_rssi']))
    else : data[i]['ap_from_ap_2_sum_ant_rssi'] = [-100.0]*120
    
    if(data[i]['sta_from_ap_0_max_ant_rssi'] != "") : data[i]['sta_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_max_ant_rssi']))
    if(data[i]['sta_from_ap_1_max_ant_rssi'] != "") : data[i]['sta_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_max_ant_rssi']))
    if(data[i]['sta_from_ap_2_max_ant_rssi'] != "") : data[i]['sta_from_ap_2_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_2_max_ant_rssi']))
    if(data[i]['sta_from_ap_0_mean_ant_rssi'] != "") : data[i]['sta_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_mean_ant_rssi']))
    if(data[i]['sta_from_ap_1_mean_ant_rssi'] != "") : data[i]['sta_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_mean_ant_rssi']))
    if(data[i]['sta_from_ap_2_mean_ant_rssi'] != "") : data[i]['sta_from_ap_2_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_2_mean_ant_rssi']))
    if(data[i]['sta_from_ap_0_sum_ant_rssi'] != "") : data[i]['sta_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_sum_ant_rssi']))
    if(data[i]['sta_from_ap_1_sum_ant_rssi'] != "") : data[i]['sta_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_sum_ant_rssi']))
    if(data[i]['sta_from_ap_2_sum_ant_rssi'] != "") : data[i]['sta_from_ap_2_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_2_sum_ant_rssi']))

    if(data[i]['sta_to_ap_0_max_ant_rssi'] != "") : data[i]['sta_to_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_max_ant_rssi']))
    if(data[i]['sta_to_ap_1_max_ant_rssi'] != "") : data[i]['sta_to_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_max_ant_rssi']))
    if(data[i]['sta_to_ap_2_max_ant_rssi'] != "") : data[i]['sta_to_ap_2_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_2_max_ant_rssi']))
    if(data[i]['sta_to_ap_0_mean_ant_rssi'] != "") : data[i]['sta_to_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_mean_ant_rssi']))
    if(data[i]['sta_to_ap_1_mean_ant_rssi'] != "") : data[i]['sta_to_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_mean_ant_rssi']))
    if(data[i]['sta_to_ap_2_mean_ant_rssi'] != "") : data[i]['sta_to_ap_2_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_2_mean_ant_rssi']))
    if(data[i]['sta_to_ap_0_sum_ant_rssi'] != "") : data[i]['sta_to_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_sum_ant_rssi']))
    if(data[i]['sta_to_ap_1_sum_ant_rssi'] != "") : data[i]['sta_to_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_sum_ant_rssi']))
    if(data[i]['sta_to_ap_2_sum_ant_rssi'] != "") : data[i]['sta_to_ap_2_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_2_sum_ant_rssi']))

for i in range(len(data)):
    if(i%3==0) :
        # print(len(data[i+1]['sta_from_ap_2_sum_ant_rssi']))
        for j in range(120):         
            folder_path = "./out/"+src_file_name+"/edge_ap_sta/test"+str(i//3)
            filename = folder_path + "/ap_sta_" + str(j).zfill(3) + ".csv"
            edges_ap_sta = []
            
            dict00 = {}
            dict00['src']=0
            dict00['dst']=0
            dict00['rssi_sum']=data[i+2]['sta_from_ap_0_sum_ant_rssi'][j]
            dict00['rssi_mean']=data[i+2]['sta_from_ap_0_mean_ant_rssi'][j]
            dict00['rssi_max']=data[i+2]['sta_from_ap_0_max_ant_rssi'][j]
            edges_ap_sta.append(dict00)
            
            dict10 = {}
            dict10['src']=1
            dict10['dst']=0
            dict10['rssi_sum']=data[i+2]['sta_from_ap_1_sum_ant_rssi'][j]
            dict10['rssi_mean']=data[i+2]['sta_from_ap_1_mean_ant_rssi'][j]
            dict10['rssi_max']=data[i+2]['sta_from_ap_1_max_ant_rssi'][j]
            edges_ap_sta.append(dict10)

            dict20 = {}
            dict20['src']=2
            dict20['dst']=0
            dict20['rssi_sum']=data[i+2]['sta_from_ap_2_sum_ant_rssi'][j]
            dict20['rssi_mean']=data[i+2]['sta_from_ap_2_mean_ant_rssi'][j]
            dict20['rssi_max']=data[i+2]['sta_from_ap_2_max_ant_rssi'][j]
            edges_ap_sta.append(dict20)

            dict01 = {}
            dict01['src']=0
            dict01['dst']=1
            dict01['rssi_sum']=data[i+1]['sta_from_ap_0_sum_ant_rssi'][j]
            dict01['rssi_mean']=data[i+1]['sta_from_ap_0_mean_ant_rssi'][j]
            dict01['rssi_max']=data[i+1]['sta_from_ap_0_max_ant_rssi'][j]
            edges_ap_sta.append(dict01)
            
            dict11 = {}
            dict11['src']=1
            dict11['dst']=1
            dict11['rssi_sum']=data[i+1]['sta_from_ap_1_sum_ant_rssi'][j]
            dict11['rssi_mean']=data[i+1]['sta_from_ap_1_mean_ant_rssi'][j]
            dict11['rssi_max']=data[i+1]['sta_from_ap_1_max_ant_rssi'][j]
            edges_ap_sta.append(dict11)

            dict21 = {}
            dict21['src']=2
            dict21['dst']=1
            dict21['rssi_sum']=data[i+1]['sta_from_ap_2_sum_ant_rssi'][j]
            dict21['rssi_mean']=data[i+1]['sta_from_ap_2_mean_ant_rssi'][j]
            dict21['rssi_max']=data[i+1]['sta_from_ap_2_max_ant_rssi'][j]
            edges_ap_sta.append(dict21)
            
            dict02 = {}
            dict02['src']=0
            dict02['dst']=2
            dict02['rssi_sum']=data[i]['sta_from_ap_0_sum_ant_rssi'][j]
            dict02['rssi_mean']=data[i]['sta_from_ap_0_mean_ant_rssi'][j]
            dict02['rssi_max']=data[i]['sta_from_ap_0_max_ant_rssi'][j]
            edges_ap_sta.append(dict02)
            
            dict12 = {}
            dict12['src']=1
            dict12['dst']=2
            dict12['rssi_sum']=data[i]['sta_from_ap_1_sum_ant_rssi'][j]
            dict12['rssi_mean']=data[i]['sta_from_ap_1_mean_ant_rssi'][j]
            dict12['rssi_max']=data[i]['sta_from_ap_1_max_ant_rssi'][j]
            edges_ap_sta.append(dict12)
            
            dict22 = {}
            dict22['src']=2
            dict22['dst']=2
            dict22['rssi_sum']=data[i]['sta_from_ap_2_sum_ant_rssi'][j]
            dict22['rssi_mean']=data[i]['sta_from_ap_2_mean_ant_rssi'][j]
            dict22['rssi_max']=data[i]['sta_from_ap_2_max_ant_rssi'][j]
            edges_ap_sta.append(dict22)

            os.makedirs(folder_path, exist_ok=True)
    
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=edges_ap_sta[0].keys())
                writer.writeheader()
                writer.writerows(edges_ap_sta)

## 两节点的 STA-AP 边特征

In [ ]:
src_file_name = 'test_set_2_2ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
# print(len(data))

for i in range(len(data)):
    if(data[i]['ap_from_ap_0_max_ant_rssi'] != "") : data[i]['ap_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_max_ant_rssi']))
    if(data[i]['ap_from_ap_1_max_ant_rssi'] != "") : data[i]['ap_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_max_ant_rssi']))
    if(data[i]['ap_from_ap_0_mean_ant_rssi'] != "") : data[i]['ap_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_mean_ant_rssi']))
    if(data[i]['ap_from_ap_1_mean_ant_rssi'] != "") : data[i]['ap_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_mean_ant_rssi']))
    if(data[i]['ap_from_ap_0_sum_ant_rssi'] != "") : data[i]['ap_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_sum_ant_rssi']))
    if(data[i]['ap_from_ap_1_sum_ant_rssi'] != "") : data[i]['ap_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_sum_ant_rssi']))
    
    if(data[i]['sta_from_ap_0_max_ant_rssi'] != "") : data[i]['sta_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_max_ant_rssi']))
    if(data[i]['sta_from_ap_1_max_ant_rssi'] != "") : data[i]['sta_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_max_ant_rssi']))
    if(data[i]['sta_from_ap_0_mean_ant_rssi'] != "") : data[i]['sta_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_mean_ant_rssi']))
    if(data[i]['sta_from_ap_1_mean_ant_rssi'] != "") : data[i]['sta_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_mean_ant_rssi']))
    if(data[i]['sta_from_ap_0_sum_ant_rssi'] != "") : data[i]['sta_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_sum_ant_rssi']))
    if(data[i]['sta_from_ap_1_sum_ant_rssi'] != "") : data[i]['sta_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_sum_ant_rssi']))

    if(data[i]['sta_to_ap_0_max_ant_rssi'] != "") : data[i]['sta_to_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_max_ant_rssi']))
    if(data[i]['sta_to_ap_1_max_ant_rssi'] != "") : data[i]['sta_to_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_max_ant_rssi']))
    if(data[i]['sta_to_ap_0_mean_ant_rssi'] != "") : data[i]['sta_to_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_mean_ant_rssi']))
    if(data[i]['sta_to_ap_1_mean_ant_rssi'] != "") : data[i]['sta_to_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_mean_ant_rssi']))
    if(data[i]['sta_to_ap_0_sum_ant_rssi'] != "") : data[i]['sta_to_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_sum_ant_rssi']))
    if(data[i]['sta_to_ap_1_sum_ant_rssi'] != "") : data[i]['sta_to_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_sum_ant_rssi']))

for i in range(len(data)):
    if(i%2==0) :
        # print(len(data[i]['ap_from_ap_0_sum_ant_rssi']))
        for j in range(120):         
            folder_path = "./out/"+src_file_name+"/edge_sta_ap/test"+str(i//2)
            filename = folder_path + "/sta_ap_" + str(j).zfill(3) + ".csv"
            edges_sta_ap = []
            
            dict00 = {}
            dict00['src']=0
            dict00['dst']=0
            dict00['rssi_sum']=data[i+1]['sta_to_ap_0_sum_ant_rssi'][j]
            dict00['rssi_mean']=data[i+1]['sta_to_ap_0_mean_ant_rssi'][j]
            dict00['rssi_max']=data[i+1]['sta_to_ap_0_max_ant_rssi'][j]
            edges_sta_ap.append(dict00)

            dict01 = {}
            dict01['src']=0
            dict01['dst']=1
            dict01['rssi_sum']=data[i+1]['sta_to_ap_1_sum_ant_rssi'][j]
            dict01['rssi_mean']=data[i+1]['sta_to_ap_1_mean_ant_rssi'][j]
            dict01['rssi_max']=data[i+1]['sta_to_ap_1_max_ant_rssi'][j]
            edges_sta_ap.append(dict01)
            
            dict10 = {}
            dict10['src']=1
            dict10['dst']=0
            dict10['rssi_sum']=data[i]['sta_to_ap_0_sum_ant_rssi'][j]
            dict10['rssi_mean']=data[i]['sta_to_ap_0_mean_ant_rssi'][j]
            dict10['rssi_max']=data[i]['sta_to_ap_0_max_ant_rssi'][j]
            edges_sta_ap.append(dict10)

            
            dict11 = {}
            dict11['src']=1
            dict11['dst']=1
            dict11['rssi_sum']=data[i]['sta_to_ap_1_sum_ant_rssi'][j]
            dict11['rssi_mean']=data[i]['sta_to_ap_1_mean_ant_rssi'][j]
            dict11['rssi_max']=data[i]['sta_to_ap_1_max_ant_rssi'][j]
            edges_sta_ap.append(dict11)

            os.makedirs(folder_path, exist_ok=True)
    
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=edges_sta_ap[0].keys())
                writer.writeheader()
                writer.writerows(edges_sta_ap)

## 三节点的 STA-AP 边特征

In [ ]:
src_file_name = 'test_set_2_3ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
# print(len(data))

for i in range(len(data)):
    if(data[i]['ap_from_ap_0_max_ant_rssi'] != "") : data[i]['ap_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_max_ant_rssi']))
    else : data[i]['ap_from_ap_0_max_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_1_max_ant_rssi'] != "") : data[i]['ap_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_max_ant_rssi']))
    else : data[i]['ap_from_ap_1_max_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_2_max_ant_rssi'] != "") : data[i]['ap_from_ap_2_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_2_max_ant_rssi']))
    else : data[i]['ap_from_ap_2_max_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_0_mean_ant_rssi'] != "") : data[i]['ap_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_mean_ant_rssi']))
    else : data[i]['ap_from_ap_0_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_1_mean_ant_rssi'] != "") : data[i]['ap_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_mean_ant_rssi']))
    else : data[i]['ap_from_ap_1_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_2_mean_ant_rssi'] != "") : data[i]['ap_from_ap_2_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_2_mean_ant_rssi']))
    else : data[i]['ap_from_ap_2_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_0_sum_ant_rssi'] != "") : data[i]['ap_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_0_sum_ant_rssi']))
    else : data[i]['ap_from_ap_0_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_1_sum_ant_rssi'] != "") : data[i]['ap_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_1_sum_ant_rssi']))
    else : data[i]['ap_from_ap_1_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['ap_from_ap_2_sum_ant_rssi'] != "") : data[i]['ap_from_ap_2_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['ap_from_ap_2_sum_ant_rssi']))
    else : data[i]['ap_from_ap_2_sum_ant_rssi'] = [-100.0]*120
    
    if(data[i]['sta_from_ap_0_max_ant_rssi'] != "") : data[i]['sta_from_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_max_ant_rssi']))
    else : data[i]['sta_from_ap_0_max_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_from_ap_1_max_ant_rssi'] != "") : data[i]['sta_from_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_max_ant_rssi']))
    else : data[i]['sta_from_ap_1_max_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_from_ap_2_max_ant_rssi'] != "") : data[i]['sta_from_ap_2_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_2_max_ant_rssi']))
    else : data[i]['sta_from_ap_2_max_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_from_ap_0_mean_ant_rssi'] != "") : data[i]['sta_from_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_mean_ant_rssi']))
    else : data[i]['sta_from_ap_0_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_from_ap_1_mean_ant_rssi'] != "") : data[i]['sta_from_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_mean_ant_rssi']))
    else : data[i]['sta_from_ap_1_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_from_ap_2_mean_ant_rssi'] != "") : data[i]['sta_from_ap_2_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_2_mean_ant_rssi']))
    else : data[i]['sta_from_ap_2_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_from_ap_0_sum_ant_rssi'] != "") : data[i]['sta_from_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_0_sum_ant_rssi']))
    else : data[i]['sta_from_ap_0_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_from_ap_1_sum_ant_rssi'] != "") : data[i]['sta_from_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_1_sum_ant_rssi']))
    else : data[i]['sta_from_ap_1_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_from_ap_2_sum_ant_rssi'] != "") : data[i]['sta_from_ap_2_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_from_ap_2_sum_ant_rssi']))
    else : data[i]['sta_from_ap_2_sum_ant_rssi'] = [-100.0]*120

    if(data[i]['sta_to_ap_0_max_ant_rssi'] != "") : data[i]['sta_to_ap_0_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_max_ant_rssi']))
    else : data[i]['sta_to_ap_0_max_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_to_ap_1_max_ant_rssi'] != "") : data[i]['sta_to_ap_1_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_max_ant_rssi']))
    else : data[i]['sta_to_ap_1_max_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_to_ap_2_max_ant_rssi'] != "") : data[i]['sta_to_ap_2_max_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_2_max_ant_rssi']))
    else : data[i]['sta_to_ap_2_max_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_to_ap_0_mean_ant_rssi'] != "") : data[i]['sta_to_ap_0_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_mean_ant_rssi']))
    else : data[i]['sta_to_ap_0_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_to_ap_1_mean_ant_rssi'] != "") : data[i]['sta_to_ap_1_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_mean_ant_rssi']))
    else : data[i]['sta_to_ap_1_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_to_ap_2_mean_ant_rssi'] != "") : data[i]['sta_to_ap_2_mean_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_2_mean_ant_rssi']))
    else : data[i]['sta_to_ap_2_mean_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_to_ap_0_sum_ant_rssi'] != "") : data[i]['sta_to_ap_0_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_0_sum_ant_rssi']))
    else : data[i]['sta_to_ap_0_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_to_ap_1_sum_ant_rssi'] != "") : data[i]['sta_to_ap_1_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_1_sum_ant_rssi']))
    else : data[i]['sta_to_ap_1_sum_ant_rssi'] = [-100.0]*120
    if(data[i]['sta_to_ap_2_sum_ant_rssi'] != "") : data[i]['sta_to_ap_2_sum_ant_rssi'] = tansRSSI(ast.literal_eval(data[i]['sta_to_ap_2_sum_ant_rssi']))
    else : data[i]['sta_to_ap_2_sum_ant_rssi'] = [-100.0]*120

for i in range(len(data)):
    if(i%3==0) :
        # print(len(data[i+1]['sta_from_ap_2_sum_ant_rssi']))
        for j in range(120):         
            folder_path = "./out/"+src_file_name+"/edge_sta_ap/test"+str(i//3)
            filename = folder_path + "/sta_ap_" + str(j).zfill(3) + ".csv"
            edges_sta_ap = []
            
            dict00 = {}
            dict00['src']=0
            dict00['dst']=0
            dict00['rssi_sum']=data[i+2]['sta_to_ap_0_sum_ant_rssi'][j]
            dict00['rssi_mean']=data[i+2]['sta_to_ap_0_mean_ant_rssi'][j]
            dict00['rssi_max']=data[i+2]['sta_to_ap_0_max_ant_rssi'][j]
            edges_sta_ap.append(dict00)
            
            dict01 = {}
            dict01['src']=0
            dict01['dst']=1
            dict01['rssi_sum']=data[i+2]['sta_to_ap_1_sum_ant_rssi'][j]
            dict01['rssi_mean']=data[i+2]['sta_to_ap_1_mean_ant_rssi'][j]
            dict01['rssi_max']=data[i+2]['sta_to_ap_1_max_ant_rssi'][j]
            edges_sta_ap.append(dict01)
            
            dict02 = {}
            dict02['src']=0
            dict02['dst']=2
            print(len(data[i+2]['sta_to_ap_2_sum_ant_rssi']))
            dict02['rssi_sum']=data[i+2]['sta_to_ap_2_sum_ant_rssi'][j]
            dict02['rssi_mean']=data[i+2]['sta_to_ap_2_mean_ant_rssi'][j]
            dict02['rssi_max']=data[i+2]['sta_to_ap_2_max_ant_rssi'][j]
            edges_sta_ap.append(dict02)

            dict10 = {}
            dict10['src']=1
            dict10['dst']=0
            dict10['rssi_sum']=data[i+1]['sta_to_ap_0_sum_ant_rssi'][j]
            dict10['rssi_mean']=data[i+1]['sta_to_ap_0_mean_ant_rssi'][j]
            dict10['rssi_max']=data[i+1]['sta_to_ap_0_max_ant_rssi'][j]
            edges_sta_ap.append(dict10)
            
            dict11 = {}
            dict11['src']=1
            dict11['dst']=1
            dict11['rssi_sum']=data[i+1]['sta_to_ap_1_sum_ant_rssi'][j]
            dict11['rssi_mean']=data[i+1]['sta_to_ap_1_mean_ant_rssi'][j]
            dict11['rssi_max']=data[i+1]['sta_to_ap_1_max_ant_rssi'][j]
            edges_sta_ap.append(dict11)           

            dict12 = {}
            dict12['src']=1
            dict12['dst']=2
            dict12['rssi_sum']=data[i+1]['sta_to_ap_2_sum_ant_rssi'][j]
            dict12['rssi_mean']=data[i+1]['sta_to_ap_2_mean_ant_rssi'][j]
            dict12['rssi_max']=data[i+1]['sta_to_ap_2_max_ant_rssi'][j]
            edges_sta_ap.append(dict12)
            
            dict20 = {}
            dict20['src']=2
            dict20['dst']=0
            dict20['rssi_sum']=data[i]['sta_to_ap_0_sum_ant_rssi'][j]
            dict20['rssi_mean']=data[i]['sta_to_ap_0_mean_ant_rssi'][j]
            dict20['rssi_max']=data[i]['sta_to_ap_0_max_ant_rssi'][j]
            edges_sta_ap.append(dict20)
            
            dict21 = {}
            dict21['src']=2
            dict21['dst']=1
            dict21['rssi_sum']=data[i]['sta_to_ap_1_sum_ant_rssi'][j]
            dict21['rssi_mean']=data[i]['sta_to_ap_1_mean_ant_rssi'][j]
            dict21['rssi_max']=data[i]['sta_to_ap_1_max_ant_rssi'][j]
            edges_sta_ap.append(dict21)                  
            
            dict22 = {}
            dict22['src']=2
            dict22['dst']=2
            dict22['rssi_sum']=data[i]['sta_to_ap_2_sum_ant_rssi'][j]
            dict22['rssi_mean']=data[i]['sta_to_ap_2_mean_ant_rssi'][j]
            dict22['rssi_max']=data[i]['sta_to_ap_2_max_ant_rssi'][j]
            edges_sta_ap.append(dict22)

            os.makedirs(folder_path, exist_ok=True)
    
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=edges_sta_ap[0].keys())
                writer.writeheader()
                writer.writerows(edges_sta_ap)

## 两节点的 sta 节点特征

In [7]:
def find_edge_by_src_dst(edge_ap_sta, src_value, dst_value):
    for edge in edge_ap_sta:
        # print(type(edge['src']),type(edge['dst']),type(src_value),type(dst_value))
        if int(edge['src']) == src_value and int(edge['dst']) == dst_value:
            return edge
    return None  # 如果未找到则返回None


In [ ]:
src_file_name = 'test_set_2_2ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
print(len(data))

for i in range(len(data)):
    if(i%2==0) :
        sta_dicts_list = []
        sta_dict1 = {}
        
        row = data[i]
        sta_dict1['type'] = 1
        sta_dict1['id'] = int(row['sta_id'][4:len(row['sta_id'])])
        sta_dict1['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        sta_dict1['protocol'] = 0
        sta_dict1['bss_id'] = row['bss_id']
        sta_dict1['pd'] = int(row['pd'])
        sta_dict1['ed'] = int(row['ed'])
        sta_dict1['nav'] = int(row['nav'])
        sta_dict1['eirp'] = 0
        sta_dict1['sinr'] = 0
        
        sta_dicts_list.append(sta_dict1)

        sta_dict = {}
        row = data[i+1]
        sta_dict['type'] = 1
        sta_dict['id'] = int(row['sta_id'][4:len(row['sta_id'])])
        sta_dict['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        sta_dict['protocol'] = 0
        sta_dict['bss_id'] = row['bss_id']
        sta_dict['pd'] = int(row['pd'])
        sta_dict['ed'] = int(row['ed'])
        sta_dict['nav'] = int(row['nav']) 
        sta_dict['eirp'] = 0
        sta_dict['sinr'] = 0
             
        sta_dicts_list.append(sta_dict)
        
        for j in range(120):
            folder_path = "./out/"+src_file_name+"/node_sta/test"+str(i//2)
            filename = folder_path + "/nodes_sta_" + str(j).zfill(3) + ".csv"
            os.makedirs(folder_path, exist_ok=True)
            
            edge_ap_sta = []
            with codecs.open('./out/'+src_file_name+'/edge_ap_sta/test'+str(i//2)+'/ap_sta_'+str(j).zfill(3)+'.csv',encoding='utf-8-sig') as f:
                for row in csv.DictReader(f,skipinitialspace=True):
                    edge_ap_sta.append(row)
            edge_ap_ap = []
            with codecs.open('./out/'+src_file_name+'/edge_ap_ap/test'+str(i//2)+'/ap_ap_'+str(j).zfill(3)+'.csv',encoding='utf-8-sig') as f:
                for row in csv.DictReader(f,skipinitialspace=True):
                    edge_ap_ap.append(row)
            
            edge_ap_sta_01 = find_edge_by_src_dst(edge_ap_sta,0,1)            
            sta_from_ap_0_sum_rssi = float(edge_ap_sta_01['rssi_sum'])
            edge_ap_sta_11 = find_edge_by_src_dst(edge_ap_sta,1,1)
            sta_from_ap_1_sum_rssi = float(edge_ap_sta_11['rssi_sum'])
            useful_rssi_list = []
            interference_rssi_list = []
            scale_factors = []
            useful_rssi_list.append(sta_from_ap_1_sum_rssi)
            interference_rssi_list.append(sta_from_ap_0_sum_rssi)
            edge_ap_ap_01 = find_edge_by_src_dst(edge_ap_ap,0,1)
            edge_ap_ap_10 = find_edge_by_src_dst(edge_ap_ap,1,0)
            ap_ap_rssi_max = 0.5*(float(edge_ap_ap_01['rssi_sum'])+float(edge_ap_ap_10['rssi_sum']))
            if (ap_ap_rssi_max > sta_dicts_list[0]['ed']) :
                scale_factors.append(0)
            elif (ap_ap_rssi_max < sta_dicts_list[0]['pd']) :
                scale_factors.append(1)
            else :
                scale_factors.append(1-(sta_dicts_list[0]['ed']-ap_ap_rssi_max)/(sta_dicts_list[0]['ed']-sta_dicts_list[0]['pd']))
            sta_dicts_list[0]['sinr'] = calculate_sinr(useful_rssi_list,interference_rssi_list,scale_factors)
            
            edge = find_edge_by_src_dst(edge_ap_sta,0,0)            
            sta_from_ap_0_sum_rssi = float(edge['rssi_sum'])
            edge = find_edge_by_src_dst(edge_ap_sta,1,0)
            sta_from_ap_1_sum_rssi = float(edge['rssi_sum'])
            useful_rssi_list = []
            interference_rssi_list = []
            useful_rssi_list.append(sta_from_ap_0_sum_rssi)
            interference_rssi_list.append(sta_from_ap_1_sum_rssi)
            edge_ap_ap_01 = find_edge_by_src_dst(edge_ap_ap,0,1)
            edge_ap_ap_10 = find_edge_by_src_dst(edge_ap_ap,1,0)
            ap_ap_rssi_max = 0.5*(float(edge_ap_ap_01['rssi_sum'])+float(edge_ap_ap_10['rssi_sum']))
            if (ap_ap_rssi_max > sta_dicts_list[0]['ed']) :
                scale_factors.append(0)
            elif (ap_ap_rssi_max < sta_dicts_list[0]['pd']) :
                scale_factors.append(1)
            else :
                scale_factors.append(1-(sta_dicts_list[0]['ed']-ap_ap_rssi_max)/(sta_dicts_list[0]['ed']-sta_dicts_list[0]['pd']))
            
            sta_dicts_list[1]['sinr'] = calculate_sinr(useful_rssi_list,interference_rssi_list,scale_factors)
            
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=sta_dicts_list[0].keys())
                writer.writeheader()
                writer.writerows(sta_dicts_list)
        sta_dicts_list.clear()
        sta_dict1.clear()
        sta_dict.clear()

## 三节点 sta 节点属性

In [ ]:
src_file_name = 'test_set_2_3ap'

data = []
with codecs.open('./Bdata/train/'+src_file_name+'.csv',encoding='utf-8-sig') as f:
    for row in csv.DictReader(f,skipinitialspace=True):
        data.append(row)
print(len(data))

for i in range(len(data)):
    if(i%3==0) :
        sta_dicts_list = []
        sta_dict1 = {}
        
        row = data[i]
        sta_dict1['type'] = 1
        # ap_dict1['id'] = row['ap_id']
        # ap_dict1['loc_id'] = row['loc_id']
        sta_dict1['id'] = int(row['sta_id'][4:len(row['sta_id'])])
        sta_dict1['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        sta_dict1['protocol'] = 0
        sta_dict1['bss_id'] = row['bss_id']
        sta_dict1['pd'] = int(row['pd'])
        sta_dict1['ed'] = int(row['ed'])
        sta_dict1['nav'] = int(row['nav'])
        sta_dict1['eirp'] = 0
        sta_dict1['sinr'] = 0
        
        sta_dicts_list.append(sta_dict1)

        sta_dict = {}
        row = data[i+1]
        sta_dict['type'] = 1
        # ap_dict1['id'] = row['ap_id']
        # ap_dict1['loc_id'] = row['loc_id']
        sta_dict['id'] = int(row['sta_id'][4:len(row['sta_id'])])
        sta_dict['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        sta_dict['protocol'] = 0
        sta_dict['bss_id'] = row['bss_id']
        sta_dict['pd'] = int(row['pd'])
        sta_dict['ed'] = int(row['ed'])
        sta_dict['nav'] = int(row['nav']) 
        sta_dict['eirp'] = 0
        sta_dict['sinr'] = 0
             
        sta_dicts_list.append(sta_dict)

        sta_dict2 = {}
        row = data[i+2]
        sta_dict2['type'] = 1
        # ap_dict1['id'] = row['ap_id']
        # ap_dict1['loc_id'] = row['loc_id']
        sta_dict2['id'] = int(row['sta_id'][4:len(row['sta_id'])])
        sta_dict2['loc_id'] = int(row['loc_id'][3:len(row['loc_id'])])
        sta_dict2['protocol'] = 0
        sta_dict2['bss_id'] = row['bss_id']
        sta_dict2['pd'] = int(row['pd'])
        sta_dict2['ed'] = int(row['ed'])
        sta_dict2['nav'] = int(row['nav']) 
        sta_dict2['eirp'] = 0
        sta_dict2['sinr'] = 0
             
        sta_dicts_list.append(sta_dict2)
        
        for j in range(120):
            folder_path = "./out/"+src_file_name+"/node_sta/test"+str(i//3)
            filename = folder_path + "/nodes_sta_" + str(j).zfill(3) + ".csv"
            os.makedirs(folder_path, exist_ok=True)
            
            edge_ap_sta = []
            with codecs.open('./out/'+src_file_name+'/edge_ap_sta/test'+str(i//3)+'/ap_sta_'+str(j).zfill(3)+'.csv',encoding='utf-8-sig') as f:
                for row in csv.DictReader(f,skipinitialspace=True):
                    edge_ap_sta.append(row)
            edge_ap_ap = []
            with codecs.open('./out/'+src_file_name+'/edge_ap_ap/test'+str(i//3)+'/ap_ap_'+str(j).zfill(3)+'.csv',encoding='utf-8-sig') as f:
                for row in csv.DictReader(f,skipinitialspace=True):
                    edge_ap_ap.append(row)
            
            edge_ap_sta_02 = find_edge_by_src_dst(edge_ap_sta,0,2)            
            sta_from_ap_0_sum_rssi = float(edge_ap_sta_02['rssi_sum'])
            edge_ap_sta_12 = find_edge_by_src_dst(edge_ap_sta,1,2)
            sta_from_ap_1_sum_rssi = float(edge_ap_sta_12['rssi_sum'])
            edge_ap_sta_22 = find_edge_by_src_dst(edge_ap_sta,2,2)
            sta_from_ap_2_sum_rssi = float(edge_ap_sta_22['rssi_sum'])
            useful_rssi_list = []
            interference_rssi_list = []
            scale_factors = []
            useful_rssi_list.append(sta_from_ap_2_sum_rssi)
            interference_rssi_list.append(sta_from_ap_0_sum_rssi)
            interference_rssi_list.append(sta_from_ap_1_sum_rssi)
            edge_ap_ap_02 = find_edge_by_src_dst(edge_ap_ap,0,2)
            edge_ap_ap_20 = find_edge_by_src_dst(edge_ap_ap,2,0)
            ap_ap_rssi_max = 0.5*(float(edge_ap_ap_02['rssi_sum'])+float(edge_ap_ap_20['rssi_sum']))
            if (ap_ap_rssi_max > sta_dicts_list[0]['ed']) :
                scale_factors.append(0)
            elif (ap_ap_rssi_max < sta_dicts_list[0]['pd']) :
                scale_factors.append(1)
            else :
                scale_factors.append(1-(sta_dicts_list[0]['ed']-ap_ap_rssi_max)/(sta_dicts_list[0]['ed']-sta_dicts_list[0]['pd']))
            edge_ap_ap_12 = find_edge_by_src_dst(edge_ap_ap,1,2)
            edge_ap_ap_21 = find_edge_by_src_dst(edge_ap_ap,2,1)
            ap_ap_rssi_max = 0.5*(float(edge_ap_ap_21['rssi_sum'])+float(edge_ap_ap_12['rssi_sum']))
            if (ap_ap_rssi_max > sta_dicts_list[0]['ed']) :
                scale_factors.append(0)
            elif (ap_ap_rssi_max < sta_dicts_list[0]['pd']) :
                scale_factors.append(1)
            else :
                scale_factors.append(1-(sta_dicts_list[0]['ed']-ap_ap_rssi_max)/(sta_dicts_list[0]['ed']-sta_dicts_list[0]['pd']))
            sta_dicts_list[0]['sinr'] = calculate_sinr(useful_rssi_list,interference_rssi_list,scale_factors)
            
            edge_ap_sta_01 = find_edge_by_src_dst(edge_ap_sta,0,1)            
            sta_from_ap_0_sum_rssi = float(edge_ap_sta_01['rssi_sum'])
            edge_ap_sta_11 = find_edge_by_src_dst(edge_ap_sta,1,1)
            sta_from_ap_1_sum_rssi = float(edge_ap_sta_11['rssi_sum'])
            edge_ap_sta_21 = find_edge_by_src_dst(edge_ap_sta,2,1)
            sta_from_ap_2_sum_rssi = float(edge_ap_sta_21['rssi_sum'])
            useful_rssi_list = []
            interference_rssi_list = []
            scale_factors = []
            useful_rssi_list.append(sta_from_ap_1_sum_rssi)
            interference_rssi_list.append(sta_from_ap_0_sum_rssi)
            interference_rssi_list.append(sta_from_ap_2_sum_rssi)
            edge_ap_ap_01 = find_edge_by_src_dst(edge_ap_ap,0,1)
            edge_ap_ap_10 = find_edge_by_src_dst(edge_ap_ap,1,0)
            ap_ap_rssi_max = 0.5*(float(edge_ap_ap_01['rssi_sum'])+float(edge_ap_ap_10['rssi_sum']))
            if (ap_ap_rssi_max > sta_dicts_list[1]['ed']) :
                scale_factors.append(0)
            elif (ap_ap_rssi_max < sta_dicts_list[1]['pd']) :
                scale_factors.append(1)
            else :
                scale_factors.append(1-(sta_dicts_list[1]['ed']-ap_ap_rssi_max)/(sta_dicts_list[1]['ed']-sta_dicts_list[1]['pd']))
            edge_ap_ap_21 = find_edge_by_src_dst(edge_ap_ap,2,1)
            edge_ap_ap_12 = find_edge_by_src_dst(edge_ap_ap,1,2)
            ap_ap_rssi_max = 0.5*(float(edge_ap_ap_21['rssi_sum'])+float(edge_ap_ap_12['rssi_sum']))
            if (ap_ap_rssi_max > sta_dicts_list[1]['ed']) :
                scale_factors.append(0)
            elif (ap_ap_rssi_max < sta_dicts_list[1]['pd']) :
                scale_factors.append(1)
            else :
                scale_factors.append(1-(sta_dicts_list[1]['ed']-ap_ap_rssi_max)/(sta_dicts_list[1]['ed']-sta_dicts_list[1]['pd']))
                
            sta_dicts_list[1]['sinr'] = calculate_sinr(useful_rssi_list,interference_rssi_list,scale_factors)
            

            
            edge = find_edge_by_src_dst(edge_ap_sta,0,0)            
            sta_from_ap_0_sum_rssi = float(edge['rssi_sum'])
            edge = find_edge_by_src_dst(edge_ap_sta,1,0)
            sta_from_ap_1_sum_rssi = float(edge['rssi_sum'])
            edge = find_edge_by_src_dst(edge_ap_sta,2,0)
            sta_from_ap_2_sum_rssi = float(edge['rssi_sum'])
            useful_rssi_list = []
            interference_rssi_list = []
            useful_rssi_list.append(sta_from_ap_0_sum_rssi)
            interference_rssi_list.append(sta_from_ap_1_sum_rssi)
            interference_rssi_list.append(sta_from_ap_2_sum_rssi)
            edge_ap_ap_10 = find_edge_by_src_dst(edge_ap_ap,1,0)
            edge_ap_ap_01 = find_edge_by_src_dst(edge_ap_ap,0,1)
            ap_ap_rssi_max = 0.5*(float(edge_ap_ap_10['rssi_sum'])+float(edge_ap_ap_01['rssi_sum']))
            if (ap_ap_rssi_max > sta_dicts_list[2]['ed']) :
                scale_factors.append(0)
            elif (ap_ap_rssi_max < sta_dicts_list[2]['pd']) :
                scale_factors.append(1)
            else :
                scale_factors.append(1-(sta_dicts_list[2]['ed']-ap_ap_rssi_max)/(sta_dicts_list[2]['ed']-sta_dicts_list[2]['pd']))
            edge_ap_ap_20 = find_edge_by_src_dst(edge_ap_ap,2,0)
            edge_ap_ap_02 = find_edge_by_src_dst(edge_ap_ap,0,2)
            ap_ap_rssi_max = 0.5*(float(edge_ap_ap_20['rssi_sum'])+float(edge_ap_ap_02['rssi_sum']))
            if (ap_ap_rssi_max > sta_dicts_list[2]['ed']) :
                scale_factors.append(0)
            elif (ap_ap_rssi_max < sta_dicts_list[2]['pd']) :
                scale_factors.append(1)
            else :
                scale_factors.append(1-(sta_dicts_list[2]['ed']-ap_ap_rssi_max)/(sta_dicts_list[2]['ed']-sta_dicts_list[2]['pd']))
            
            sta_dicts_list[2]['sinr'] = calculate_sinr(useful_rssi_list,interference_rssi_list,scale_factors)
            
            with open(filename, mode='w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=sta_dicts_list[0].keys())
                writer.writeheader()
                writer.writerows(sta_dicts_list)
        sta_dicts_list.clear()
        sta_dict1.clear()
        sta_dict.clear()

# DGL 图构建

## 训练图数据 v1

In [ ]:
table_name = 'training_set_2ap_loc0_nav86'
top_folder = './out/'+table_name+'/'
sub_folder = 'edge_ap_ap/'

ap_num = 2

edge_ap_ap_path=[]
edge_ap_sta_path=[]
edge_sta_ap_path=[]

for i in range(120):
    file_name = 'ap_ap_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_ap_ap_path.append(files)

sub_folder = 'edge_ap_sta/'
for i in range(120):
    file_name = 'ap_sta_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_ap_sta_path.append(files)

sub_folder = 'edge_sta_ap/'
for i in range(120):
    file_name = 'sta_ap_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_sta_ap_path.append(files)

node_ap_path=[]
node_sta_path=[]

sub_folder = 'node_ap/'
for i in range(120):
    file_name = 'nodes_ap_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    node_ap_path.append(files)

sub_folder = 'node_sta/'
for i in range(120):
    file_name = 'nodes_sta_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    node_sta_path.append(files)

ap_predict_path = []
sub_folder = 'node_ap_predict/'
for i in range(120):
    file_name = 'nodes_ap_predict_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    ap_predict_path.append(files)


# 每个数据表得到120个graph
for i in range(120):
    # 得到 ap-ap 边
    ap_ap_src = []
    ap_ap_dst = []
    ap_ap_feat = []
    # 得到 ap-sta 边
    ap_sta_src = []
    ap_sta_dst = []
    ap_sta_feat = []
    # 得到 sta-ap 边
    sta_ap_src = []
    sta_ap_dst = []
    sta_ap_feat = []
    # 得到ap节点的特征
    ap_feat = []
    # 得到sta节点的特征
    sta_feat = []
    # ap的预测结果
    ap_predict = []
    # 每个 graph 包含 test_count 张图
    for j in range(len(edge_ap_ap_path[i])): 
        with codecs.open(edge_ap_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                ap_ap_src.append(row['src'])
                ap_ap_dst.append(row['dst'])
                ap_ap_feat.append(row)
        with codecs.open(edge_ap_sta_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                ap_sta_src.append(row['src'])
                ap_sta_dst.append(row['dst'])
                ap_sta_feat.append(row)
        with codecs.open(edge_sta_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                sta_ap_src.append(row['src'])
                sta_ap_dst.append(row['dst'])
                sta_ap_feat.append(row)
        with codecs.open(node_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                ap_feat.append(row)
        with codecs.open(node_sta_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                sta_feat.append(row)
        with codecs.open(ap_predict_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                ap_predict.append(row)
    
    # 异构图的边集
    edge_dict = {
        ('ap', 'ap-ap', 'ap'): (torch.tensor(ap_ap_src), torch.tensor(ap_ap_dst)),
        ('ap', 'ap-sta', 'sta'): (torch.tensor(ap_sta_src), torch.tensor(ap_sta_dst)),
        ('sta', 'sta-ap', 'ap'): (torch.tensor(sta_ap_src), torch.tensor(sta_ap_dst))
    }
    g = dgl.heterograph(edge_dict)
    # 异构图的节点预测集
    ap_predict.sort(key=lambda x: x['id'])
    
    # 异构图的节点特征集
    ap_feat.sort(key=lambda x: x['id'])
    sta_feat.sort(key=lambda x: x['id'])
    
    # 把预测集中的 nss mcs 放进特征集
    predict_dict = {item['id']:item for item in ap_predict}
    for feat in ap_feat:
        ap_id = feat['id']
        feat['mcs'] = predict_dict[ap_id]['mcs']
        feat['nss'] = predict_dict[ap_id]['nss']
    for feat in sta_feat:
        sta_id = feat['id']
        feat['mcs'] = predict_dict[sta_id]['mcs']
        feat['nss'] = predict_dict[sta_id]['nss']
        
    ap_feat = [list(row.values()) for row in ap_feat]
    sta_feat = [list(row.values()) for row in sta_feat]
    
    node_feature_dict = {
        'ap': torch.tensor(ap_feat),
        'sta': torch.tensor(sta_feat)
    }

    ap_predict = [list(row.values()) for row in ap_predict]
    node_predict_dict = {
        'ap': torch.tensor(ap_predict),
        'sta': torch.tensor(ap_predict)
    }
    
    # 异构图的边特征集
    ap_ap_feat = [list(row.values()) for row in ap_ap_feat]
    ap_sta_feat = [list(row.values()) for row in ap_sta_feat]
    sta_ap_feat = [list(row.values()) for row in sta_ap_feat]
    edge_feature_dict = {
        ('ap', 'ap-ap', 'ap'): torch.tensor(ap_ap_feat),
        ('ap', 'ap-sta', 'sta'): torch.tensor(ap_sta_feat),
        ('sta', 'sta-ap', 'ap'): torch.tensor(sta_ap_feat)
    }
    # 构造DGL异构图
    mask_dict = {
        'ap' : torch.ones(len(ap_feat),1),
        'sta' : torch.zeros(len(sta_feat),1)
    }
    g.ndata['feat'] = node_feature_dict
    g.ndata['predict'] = node_predict_dict
    g.ndata['mask'] = mask_dict
    g.edata['feat'] = edge_feature_dict

    folder_pth = './HTNetDataset/'+table_name
    file_name = folder_pth+'/train_'+str(i)+'.bin'
    os.makedirs(folder_pth, exist_ok=True)
    dgl.save_graphs(file_name, g)


## 测试图数据 v1

In [ ]:
table_name = 'test_set_2_3ap'
top_folder = './out/'+table_name+'/'
sub_folder = 'edge_ap_ap/'

ap_num = 3

edge_ap_ap_path=[]
edge_ap_sta_path=[]
edge_sta_ap_path=[]

for i in range(120):
    file_name = 'ap_ap_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_ap_ap_path.append(files)

sub_folder = 'edge_ap_sta/'
for i in range(120):
    file_name = 'ap_sta_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_ap_sta_path.append(files)

sub_folder = 'edge_sta_ap/'
for i in range(120):
    file_name = 'sta_ap_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_sta_ap_path.append(files)

node_ap_path=[]
node_sta_path=[]

sub_folder = 'node_ap/'
for i in range(120):
    file_name = 'nodes_ap_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    node_ap_path.append(files)

sub_folder = 'node_sta/'
for i in range(120):
    file_name = 'nodes_sta_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    node_sta_path.append(files)

ap_predict_path = []
sub_folder = 'node_ap_predict/'
for i in range(120):
    file_name = 'nodes_ap_predict_'+str(i).zfill(3)+'.csv'
    files = []
    length = len(os.listdir(top_folder+sub_folder))
    for j in range(length):
        folder_pth = "test"+str(j)
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    ap_predict_path.append(files)


# 每个数据表得到120个graph
for i in range(120):
    # 得到 ap-ap 边
    ap_ap_src = []
    ap_ap_dst = []
    ap_ap_feat = []
    # 得到 ap-sta 边
    ap_sta_src = []
    ap_sta_dst = []
    ap_sta_feat = []
    # 得到 sta-ap 边
    sta_ap_src = []
    sta_ap_dst = []
    sta_ap_feat = []
    # 得到ap节点的特征
    ap_feat = []
    # 得到sta节点的特征
    sta_feat = []
    # ap的预测结果
    ap_predict = []
    # 每个 graph 包含 test_count 张图
    for j in range(len(edge_ap_ap_path[i])): 
        with codecs.open(edge_ap_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                ap_ap_src.append(row['src'])
                ap_ap_dst.append(row['dst'])
                ap_ap_feat.append(row)
        with codecs.open(edge_ap_sta_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                ap_sta_src.append(row['src'])
                ap_sta_dst.append(row['dst'])
                ap_sta_feat.append(row)
        with codecs.open(edge_sta_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                sta_ap_src.append(row['src'])
                sta_ap_dst.append(row['dst'])
                sta_ap_feat.append(row)
        with codecs.open(node_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                ap_feat.append(row)
        with codecs.open(node_sta_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                sta_feat.append(row)
        with codecs.open(ap_predict_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                ap_predict.append(row)
    
    # 异构图的边集
    edge_dict = {
        ('ap', 'ap-ap', 'ap'): (torch.tensor(ap_ap_src), torch.tensor(ap_ap_dst)),
        ('ap', 'ap-sta', 'sta'): (torch.tensor(ap_sta_src), torch.tensor(ap_sta_dst)),
        ('sta', 'sta-ap', 'ap'): (torch.tensor(sta_ap_src), torch.tensor(sta_ap_dst))
    }
    g = dgl.heterograph(edge_dict)
    # 异构图的节点预测集
    ap_predict.sort(key=lambda x: x['id'])
    
    # 异构图的节点特征集
    ap_feat.sort(key=lambda x: x['id'])
    sta_feat.sort(key=lambda x: x['id'])
    
    # 把预测集中的 nss mcs 放进特征集
    predict_dict = {item['id']:item for item in ap_predict}
    for feat in ap_feat:
        ap_id = feat['id']
        feat['mcs'] = predict_dict[ap_id]['mcs']
        feat['nss'] = predict_dict[ap_id]['nss']
    for feat in sta_feat:
        sta_id = feat['id']
        feat['mcs'] = predict_dict[sta_id]['mcs']
        feat['nss'] = predict_dict[sta_id]['nss']
        
    ap_feat = [list(row.values()) for row in ap_feat]
    sta_feat = [list(row.values()) for row in sta_feat]
    
    node_feature_dict = {
        'ap': torch.tensor(ap_feat),
        'sta': torch.tensor(sta_feat)
    }

    node_predict_dict = {
        'ap': torch.zeros(len(ap_feat),9),
        'sta': torch.zeros(len(ap_feat),9)
    }
    
    # 异构图的边特征集
    ap_ap_feat = [list(row.values()) for row in ap_ap_feat]
    ap_sta_feat = [list(row.values()) for row in ap_sta_feat]
    sta_ap_feat = [list(row.values()) for row in sta_ap_feat]
    edge_feature_dict = {
        ('ap', 'ap-ap', 'ap'): torch.tensor(ap_ap_feat),
        ('ap', 'ap-sta', 'sta'): torch.tensor(ap_sta_feat),
        ('sta', 'sta-ap', 'ap'): torch.tensor(sta_ap_feat)
    }
    # 构造DGL异构图
    mask_dict = {
        'ap' : torch.ones(len(ap_feat),1),
        'sta' : torch.zeros(len(sta_feat),1)
    }
    g.ndata['feat'] = node_feature_dict
    g.ndata['predict'] = node_predict_dict
    g.ndata['mask'] = mask_dict
    g.edata['feat'] = edge_feature_dict

    folder_pth = './HTNetDataset/'+table_name
    file_name = folder_pth+'/train_'+str(i)+'.bin'
    os.makedirs(folder_pth, exist_ok=True)
    dgl.save_graphs(file_name, g)


## 训练图数据 v2

In [ ]:
table_name = 'training_set_2ap_loc0_nav86'
top_folder = './out/'+table_name+'/'
sub_folder = 'edge_ap_ap/'

ap_num = 2

edge_ap_ap_path=[]
edge_ap_sta_path=[]
edge_sta_ap_path=[]

length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'ap_ap_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_ap_ap_path.append(files)

sub_folder = 'edge_ap_sta/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'ap_sta_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_ap_sta_path.append(files)


sub_folder = 'edge_sta_ap/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'sta_ap_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_sta_ap_path.append(files)

node_ap_path=[]
node_sta_path=[]

sub_folder = 'node_ap/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'nodes_ap_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    node_ap_path.append(files)

sub_folder = 'node_sta/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'nodes_sta_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    node_sta_path.append(files)

ap_predict_path = []
sub_folder = 'node_ap_predict/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'nodes_ap_predict_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    ap_predict_path.append(files)

print(edge_ap_ap_path)
# 每个数据表得到120个graph
for i in range(len(edge_ap_ap_path)):
    # 得到 ap-ap 边
    ap_ap_src = []
    ap_ap_dst = []
    ap_ap_feat = []
    # 得到 ap-sta 边
    ap_sta_src = []
    ap_sta_dst = []
    ap_sta_feat = []
    # 得到 sta-ap 边
    sta_ap_src = []
    sta_ap_dst = []
    sta_ap_feat = []
    # 得到ap节点的特征
    ap_feat = []
    # 得到sta节点的特征
    sta_feat = []
    # ap的预测结果
    ap_predict = []
    # 每个 graph 包含 test_count 张图
    for j in range(len(edge_ap_ap_path[i])): 
        with codecs.open(edge_ap_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                ap_ap_src.append(row['src'])
                ap_ap_dst.append(row['dst'])
                ap_ap_feat.append(row)
        with codecs.open(edge_ap_sta_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                ap_sta_src.append(row['src'])
                ap_sta_dst.append(row['dst'])
                ap_sta_feat.append(row)
        with codecs.open(edge_sta_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                sta_ap_src.append(row['src'])
                sta_ap_dst.append(row['dst'])
                sta_ap_feat.append(row)
        with codecs.open(node_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                ap_feat.append(row)
        with codecs.open(node_sta_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                sta_feat.append(row)
        with codecs.open(ap_predict_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                ap_predict.append(row)
    
    # 异构图的边集
    edge_dict = {
        ('ap', 'ap-ap', 'ap'): (torch.tensor(ap_ap_src), torch.tensor(ap_ap_dst)),
        ('ap', 'ap-sta', 'sta'): (torch.tensor(ap_sta_src), torch.tensor(ap_sta_dst)),
        ('sta', 'sta-ap', 'ap'): (torch.tensor(sta_ap_src), torch.tensor(sta_ap_dst))
    }
    g = dgl.heterograph(edge_dict)
    # 异构图的节点预测集
    ap_predict.sort(key=lambda x: x['id'])
    
    # 异构图的节点特征集
    ap_feat.sort(key=lambda x: x['id'])
    sta_feat.sort(key=lambda x: x['id'])
    
    # 把预测集中的 nss mcs 放进特征集
    predict_dict = {item['id']:item for item in ap_predict}
    for feat in ap_feat:
        ap_id = feat['id']
        feat['mcs'] = predict_dict[ap_id]['mcs']
        feat['nss'] = predict_dict[ap_id]['nss']
    for feat in sta_feat:
        sta_id = feat['id']
        feat['mcs'] = predict_dict[sta_id]['mcs']
        feat['nss'] = predict_dict[sta_id]['nss']
        
    ap_feat = [list(row.values()) for row in ap_feat]
    sta_feat = [list(row.values()) for row in sta_feat]
    
    node_feature_dict = {
        'ap': torch.tensor(ap_feat),
        'sta': torch.tensor(sta_feat)
    }

    ap_predict = [list(row.values()) for row in ap_predict]
    node_predict_dict = {
        'ap': torch.tensor(ap_predict),
        'sta': torch.tensor(ap_predict)
    }
    
    # 异构图的边特征集
    ap_ap_feat = [list(row.values()) for row in ap_ap_feat]
    ap_sta_feat = [list(row.values()) for row in ap_sta_feat]
    sta_ap_feat = [list(row.values()) for row in sta_ap_feat]
    edge_feature_dict = {
        ('ap', 'ap-ap', 'ap'): torch.tensor(ap_ap_feat),
        ('ap', 'ap-sta', 'sta'): torch.tensor(ap_sta_feat),
        ('sta', 'sta-ap', 'ap'): torch.tensor(sta_ap_feat)
    }
    # 构造DGL异构图
    mask_dict = {
        'ap' : torch.ones(len(ap_feat),1),
        'sta' : torch.zeros(len(sta_feat),1)
    }
    g.ndata['feat'] = node_feature_dict
    g.ndata['predict'] = node_predict_dict
    g.ndata['mask'] = mask_dict
    g.edata['feat'] = edge_feature_dict

    folder_pth = './HTNetDataset/'+table_name
    file_name = folder_pth+'/train_'+str(i)+'.bin'
    os.makedirs(folder_pth, exist_ok=True)
    dgl.save_graphs(file_name, g)


## 测试图数据 v2

In [ ]:
table_name = 'test_set_2_2ap'
top_folder = './out/'+table_name+'/'
sub_folder = 'edge_ap_ap/'

ap_num = 2

edge_ap_ap_path=[]
edge_ap_sta_path=[]
edge_sta_ap_path=[]

length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'ap_ap_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_ap_ap_path.append(files)

sub_folder = 'edge_ap_sta/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'ap_sta_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_ap_sta_path.append(files)


sub_folder = 'edge_sta_ap/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'sta_ap_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    edge_sta_ap_path.append(files)

node_ap_path=[]
node_sta_path=[]

sub_folder = 'node_ap/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'nodes_ap_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    node_ap_path.append(files)

sub_folder = 'node_sta/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'nodes_sta_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    node_sta_path.append(files)

ap_predict_path = []
sub_folder = 'node_ap_predict/'
length = len(os.listdir(top_folder+sub_folder))
for i in range(length):
    files = []
    folder_pth = "test"+str(i)
    for j in range(120):
        file_name = 'nodes_ap_predict_'+str(j).zfill(3)+'.csv'
        files.append(top_folder+sub_folder+folder_pth+'/'+file_name)
    ap_predict_path.append(files)

print(ap_predict_path)
# 每个数据表得到120个graph
for i in range(len(edge_ap_ap_path)):
    # 得到 ap-ap 边
    ap_ap_src = []
    ap_ap_dst = []
    ap_ap_feat = []
    # 得到 ap-sta 边
    ap_sta_src = []
    ap_sta_dst = []
    ap_sta_feat = []
    # 得到 sta-ap 边
    sta_ap_src = []
    sta_ap_dst = []
    sta_ap_feat = []
    # 得到ap节点的特征
    ap_feat = []
    # 得到sta节点的特征
    sta_feat = []
    # ap的预测结果
    ap_predict = []
    # 每个 graph 包含 test_count 张图
    for j in range(len(edge_ap_ap_path[i])): 
        with codecs.open(edge_ap_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                ap_ap_src.append(row['src'])
                ap_ap_dst.append(row['dst'])
                ap_ap_feat.append(row)
        with codecs.open(edge_ap_sta_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                ap_sta_src.append(row['src'])
                ap_sta_dst.append(row['dst'])
                ap_sta_feat.append(row)
        with codecs.open(edge_sta_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['src'] = int(row['src'])+j*ap_num
                row['dst'] = int(row['dst'])+j*ap_num
                sta_ap_src.append(row['src'])
                sta_ap_dst.append(row['dst'])
                sta_ap_feat.append(row)
        with codecs.open(node_ap_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                ap_feat.append(row)
        with codecs.open(node_sta_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                sta_feat.append(row)
        with codecs.open(ap_predict_path[i][j], 'r', encoding='utf-8-sig') as f:
            for row in csv.DictReader(f,skipinitialspace=True):
                print(row)
                row = {key: np.float32(value) for key, value in row.items()}
                row['id'] = row['id']+j*ap_num
                ap_predict.append(row)
    
    # 异构图的边集
    edge_dict = {
        ('ap', 'ap-ap', 'ap'): (torch.tensor(ap_ap_src), torch.tensor(ap_ap_dst)),
        ('ap', 'ap-sta', 'sta'): (torch.tensor(ap_sta_src), torch.tensor(ap_sta_dst)),
        ('sta', 'sta-ap', 'ap'): (torch.tensor(sta_ap_src), torch.tensor(sta_ap_dst))
    }
    g = dgl.heterograph(edge_dict)
    # 异构图的节点预测集
    ap_predict.sort(key=lambda x: x['id'])
    
    # 异构图的节点特征集
    ap_feat.sort(key=lambda x: x['id'])
    sta_feat.sort(key=lambda x: x['id'])
    
    # 把预测集中的 nss mcs 放进特征集
    predict_dict = {item['id']:item for item in ap_predict}
    for feat in ap_feat:
        ap_id = feat['id']
        feat['mcs'] = predict_dict[ap_id]['mcs']
        feat['nss'] = predict_dict[ap_id]['nss']
    for feat in sta_feat:
        sta_id = feat['id']
        feat['mcs'] = predict_dict[sta_id]['mcs']
        feat['nss'] = predict_dict[sta_id]['nss']
        
    ap_feat = [list(row.values()) for row in ap_feat]
    sta_feat = [list(row.values()) for row in sta_feat]
    
    node_feature_dict = {
        'ap': torch.tensor(ap_feat),
        'sta': torch.tensor(sta_feat)
    }

    node_predict_dict = {
        'ap': torch.zeros(len(ap_feat),9),
        'sta': torch.zeros(len(ap_feat),9)
    }
    
    # 异构图的边特征集
    ap_ap_feat = [list(row.values()) for row in ap_ap_feat]
    ap_sta_feat = [list(row.values()) for row in ap_sta_feat]
    sta_ap_feat = [list(row.values()) for row in sta_ap_feat]
    edge_feature_dict = {
        ('ap', 'ap-ap', 'ap'): torch.tensor(ap_ap_feat),
        ('ap', 'ap-sta', 'sta'): torch.tensor(ap_sta_feat),
        ('sta', 'sta-ap', 'ap'): torch.tensor(sta_ap_feat)
    }
    # 构造DGL异构图
    mask_dict = {
        'ap' : torch.ones(len(ap_feat),1),
        'sta' : torch.zeros(len(sta_feat),1)
    }
    g.ndata['feat'] = node_feature_dict
    g.ndata['predict'] = node_predict_dict
    g.ndata['mask'] = mask_dict
    g.edata['feat'] = edge_feature_dict

    folder_pth = './HTNetDataset/'+table_name
    file_name = folder_pth+'/train_'+str(i)+'.bin'
    os.makedirs(folder_pth, exist_ok=True)
    dgl.save_graphs(file_name, g)
